In [ ]:
import requests       # pour lancer une requête web (module natif)
import pandas as pd   # pour stocker et manipuler les données requêtées

# # Exemple avec l’API BAN:
root = 'https://api-adresse.data.gouv.fr/'
endpoint = 'search/'
key = '?q='
address = '8+bd+du+port'

query = f'{root}{endpoint}{key}{address}'

print(query)

In [ ]:
req = requests.get(query)  # requête web

# # Récupération de l'ensemble de la donnée:
data = req.json()
df2 = pd.DataFrame(data)
df2.head()  # observez le résultat...

In [ ]:
df3 = pd.DataFrame(data['features'])
df3.head() # observez le résultat...

In [ ]:
# # Lorsque la structure JSON de sortie est emboitée comme ici, utiliser `pd.json_normalize`:
df4 = pd.json_normalize(data['features'])
df4.head() # observez le résultat...

In [ ]:
!pip install folium

In [ ]:
# # Récupérer des coordonnées avec l'API BAN:
import requests

postcode = 92120
url_ban_example = f'https://api-adresse.data.gouv.fr/search/?q={address.replace(" ", "+")}&postcode={postcode}'
req = requests.get(url_ban_example)

# req.json()
prop = req.json().get('features')[0].get('properties') 
x, y = prop.get('x'), prop.get('y') 
print(f'x: {x}, y: {y}')

# # Afficher les résultats sur une carte avec `GeoPandas` et `Folium`
import geopandas as gpd
import folium

gdf = gpd.GeoDataFrame(data=pd.DataFrame({'ville': ['Montrouge']}), geometry=gpd.points_from_xy([x], [y]), crs='EPSG:2154')
gdf.to_crs(epsg=4326, inplace=True)
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y

center = gdf[['latitude', 'longitude']].mean().values.tolist()
# sw = gdf[['latitude', 'longitude']].min().values.tolist()
# ne = gdf[['latitude', 'longitude']].max().values.tolist()

m = folium.Map(location = center, tiles='CartoDB.Positron') # CartoDB.Positron, openstreetmap 

# I can add marker one by one on the map
for i in range(0,len(gdf)):
    folium.Marker(
        [gdf.iloc[i]['latitude'], gdf.iloc[i]['longitude']],
        popup=gdf.iloc[i]['ville']
    ).add_to(m)

# m.fit_bounds([sw, ne])
m